In [ ]:
#github up用
#日本時間で特定の時間になったら発注（ビットフライヤー用）
#個人が勉強用につくってます。動作保証はありません。

import requests
import json
import time
import hmac
import hashlib
from datetime import datetime, timezone, timedelta


url = "https://api.bitflyer.com"
path_url = "/v1/me/sendchildorder"
method = "POST"


# 指定した時刻の年月日を現在時刻から取得する
JST = timezone(timedelta(hours=+9), 'JST')                                 # JST (日本標準時) のタイムゾーンオブジェクトを作成する
now = datetime.now(JST)                                                    # 現在時刻を取得し、JST に変換する
order_time = datetime(now.year, now.month, now.day, 18, 0, 0, tzinfo=JST)  # 日本時間を指定する


# 指定した時刻になるまで待機する
while now < order_time:
    now = datetime.now()
    time.sleep(30)


#購入注文を出す Lightning FXにて
timestamp = int(time.time() * 1000)
timestamp = str(timestamp)

payload = {
    "product_code": "FX_BTC_JPY",
    "child_order_type": "LIMIT",  #指値注文の場合は "LIMIT", 成行注文の場合は "MARKET"
    "side": "BUY",                #買い注文の場合は "BUY", 売り注文の場合は "SELL" 
    "price": price-20000,             #child_order_type に "LIMIT" を指定した場合は必須
    "size": 0.01,                 #注文数量を指定します。
    "minute_to_expire": 10000,    #期限切れまでの時間を分で指定します。省略した場合の値は 43200 (30 日間) です。
    "time_in_force": "GTC"        # 執行数量条件 を "GTC", "IOC", "FOK" のいずれかで指定します。省略した場合の値は "GTC" です。
}


body = json.dumps(payload) #payloadをjson形式へ変換
message = timestamp + method + path_url + body
signature = hmac.new(bytearray('secrit API'.encode('utf-8')), message.encode('utf-8') , digestmod = hashlib.sha256 ).hexdigest()

headers = {
        'ACCESS-KEY' : 'API key',
        'ACCESS-TIMESTAMP' : timestamp,
        'ACCESS-SIGN' : signature,
        'Content-Type' : 'application/json'}

response = requests.post( url + path_url , headers = headers, data=body)

print(response.text)



